In [1]:
import json
import time
import pymongo
import pandas as pd
from kubernetes import client,config,watch

In [2]:
config.load_kube_config()
api = client.CustomObjectsApi()

In [4]:
mongoclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = mongoclient['resource']

In [5]:
for i in range(60):
    data = api.list_namespaced_custom_object(group="metrics.k8s.io",version="v1beta1", namespace="default", plural="pods")['items']
    for i in data:
        timeline = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        collection = db[i['containers'][0]['name']]
        item = dict({'time':timeline}, **i['containers'][0]['usage'])
        collection.insert_one(item)
    time.sleep(1)